<a href="https://colab.research.google.com/github/BrotherKim/AI599/blob/master/231228_prisionmoney_money_hak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIST AI599 법률인공지능

In [ ]:
# train variables
TARGET = 'money'
TARGET_SET = 'avail' # all, avail, nonavail. nonavail의 경우 법령이 파싱안되는 문제 발생.
TARGET_START = 0 # 0 if you want zero values, INCLUDE THST VALUE
TARGET_END = 36 # INCLUDE THIS VALUEㅠㅁㅠㅁㄳ
TARGET_APPROX = 0 # '2'개월 또는 '1000000'원
TARGET_OPT = 'binary' # class, binary
TRAIN_COLUMN = 'fy' # raw, facts, statutes, yh, fy
MODEL_DIR = '/content/drive/MyDrive/AI599/models'

## Google drive connect.
connect google drive storage for saving checkpoints.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets==2.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 11.5 MB/s eta 0:00:00


In [ ]:
!pip install --quiet transformers==4.22.0
!pip install --quiet sentencepiece==0.1.97
!pip install --quiet pytorch-lightning==1.7.6 torchmetrics==0.11.4
!pip install --quiet sklearn
!pip install --quiet torchsampler
!pip install --quiet torchmetrics

# !pip install --quiet pytorch>=1.10.0
# !pip install --quiet transformers==4.16.2
# !pip install --quiet pytorch-lightning==1.5.10
!pip install --quiet streamlit==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.5/707.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 14.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from functools import reduce
import random

import torch
import numpy as np
import transformers
import pytorch_lightning as pl
from transformers import AutoTokenizer, BertForSequenceClassification

import pandas as pd
import datasets

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

## 학성이 데이터 읽어와서 준비

In [ ]:
dataset = {}

In [ ]:
hak_file_path = '/content/drive/MyDrive/FinalData/1130_2004001000_F.csv'
df = pd.read_csv(hak_file_path)

In [ ]:
df

,Unnamed: 0,instruction,output
0,0,레인지로버 스포츠 SDV6 승용차의 운전업무에 종사하는 피고인은 2021. 5. 1...,벌금
1,1,수원지방법원 성남지원에서 아동.청소년의성보호에관한법률위반(강간등)죄 등으로 소년부 ...,벌금
2,2,2020. 1. 24. 12:00경 혈중알코올농도 0.104%의 술에 취한 상태로 ...,벌금
3,3,피고인은 BMW 520d 승용차를 운전하는 업무에 종사하는 사람에게는 전방 및 좌우...,벌금
4,4,2021 6. 24. 19:35경 대전 대덕구 B에 있는 ‘C’ 앞 편도 1차선 도...,벌금
...,...,...,...
4643,4643,피고인은 2021. 7. 17. 02:10경 창원시 마산합포구 B에 있는 'C' 앞...,징역
4644,4644,충북 영동군 B 앞 도로부터 충북 영동군 C에 있는 D 앞 도로까지 약 2km 구간...,벌금
4645,4645,2011. 2. 25. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄로 벌금 ...,징역
4646,4646,피고인은 2009. 7. 16. 대전지방법원 서산지원에서 도로교통법위반(음주운전)죄...,징역


In [ ]:
# prompt: df를 순회하면서 새로운 hak_df라는 dataframe을 생성할건데, output이 징역이면 prison에 1을, 집행유예면 yooye에 1을 넣고 instruction은 fy에 넣어줘

hak_df = pd.DataFrame()

for i in range(len(df)):
  row = df.iloc[i]
  hak_df.loc[i, 'id'] = i
  hak_df.loc[i, 'yooye'] = 0
  hak_df.loc[i, 'prison'] = 0
  hak_df.loc[i, 'money'] = 0
  hak_df.loc[i, 'fy'] = row['instruction']
  if row['output'] == '징역':
    hak_df.loc[i, 'prison'] = 1
  elif row['output'] == '집행유예':
    hak_df.loc[i, 'yooye'] = 1
  elif row['output'] == '벌금':
    hak_df.loc[i, 'money'] = 1


In [ ]:
hak_df

,id,yooye,prison,money,fy
0,0.0,0.0,0.0,1.0,레인지로버 스포츠 SDV6 승용차의 운전업무에 종사하는 피고인은 2021. 5. 1...
1,1.0,0.0,0.0,1.0,수원지방법원 성남지원에서 아동.청소년의성보호에관한법률위반(강간등)죄 등으로 소년부 ...
2,2.0,0.0,0.0,1.0,2020. 1. 24. 12:00경 혈중알코올농도 0.104%의 술에 취한 상태로 ...
3,3.0,0.0,0.0,1.0,피고인은 BMW 520d 승용차를 운전하는 업무에 종사하는 사람에게는 전방 및 좌우...
4,4.0,0.0,0.0,1.0,2021 6. 24. 19:35경 대전 대덕구 B에 있는 ‘C’ 앞 편도 1차선 도...
...,...,...,...,...,...
4643,4643.0,0.0,1.0,0.0,피고인은 2021. 7. 17. 02:10경 창원시 마산합포구 B에 있는 'C' 앞...
4644,4644.0,0.0,0.0,1.0,충북 영동군 B 앞 도로부터 충북 영동군 C에 있는 D 앞 도로까지 약 2km 구간...
4645,4645.0,0.0,1.0,0.0,2011. 2. 25. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄로 벌금 ...
4646,4646.0,0.0,1.0,0.0,피고인은 2009. 7. 16. 대전지방법원 서산지원에서 도로교통법위반(음주운전)죄...


In [ ]:
dataset['avail'] = hak_df

## 전처리된 데이터로 학습.
모델이 학습할 수 있도록 데이터의 형태를 변경해 학습합니다.

In [ ]:
# https://data-newbie.tistory.com/721
from torch.utils.data import Sampler
class OverSampler(Sampler):
    """Over Sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, class_vector, batch_size):
        """
        Arguments
        ---------
        class_vector : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(class_vector.size(0) / batch_size)
        self.class_vector = class_vector
        self.indices = list(range(len(self.class_vector)))
        self.batch_size =batch_size
        uni_label = torch.unique(class_vector)
        uni_label_sorted, _  = torch.sort(uni_label)
        print(uni_label_sorted)
        uni_label_sorted = uni_label_sorted.detach().numpy()
        label_bin = torch.bincount(class_vector.int()).detach().numpy()
        label_to_count = dict(zip(uni_label_sorted , label_bin))
        weights = [ len(class_vector) / label_to_count[float(label)] for label in           class_vector]
        self.weights = torch.DoubleTensor(weights)

    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.batch_size, replacement=True))

    def __len__(self):
        return len(self.class_vector)

In [ ]:
#trainsampler = OverSampler(class_vector=torch.from_numpy(label.values.squeeze()), batch_size=batch_size)
#trainloader = DataLoader(traindataset, batch_size=batch_size, shuffle=False,


In [ ]:
dataset['avail']

,id,yooye,prison,money,fy
0,0.0,0.0,0.0,1.0,레인지로버 스포츠 SDV6 승용차의 운전업무에 종사하는 피고인은 2021. 5. 1...
1,1.0,0.0,0.0,1.0,수원지방법원 성남지원에서 아동.청소년의성보호에관한법률위반(강간등)죄 등으로 소년부 ...
2,2.0,0.0,0.0,1.0,2020. 1. 24. 12:00경 혈중알코올농도 0.104%의 술에 취한 상태로 ...
3,3.0,0.0,0.0,1.0,피고인은 BMW 520d 승용차를 운전하는 업무에 종사하는 사람에게는 전방 및 좌우...
4,4.0,0.0,0.0,1.0,2021 6. 24. 19:35경 대전 대덕구 B에 있는 ‘C’ 앞 편도 1차선 도...
...,...,...,...,...,...
4643,4643.0,0.0,1.0,0.0,피고인은 2021. 7. 17. 02:10경 창원시 마산합포구 B에 있는 'C' 앞...
4644,4644.0,0.0,0.0,1.0,충북 영동군 B 앞 도로부터 충북 영동군 C에 있는 D 앞 도로까지 약 2km 구간...
4645,4645.0,0.0,1.0,0.0,2011. 2. 25. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄로 벌금 ...
4646,4646.0,0.0,1.0,0.0,피고인은 2009. 7. 16. 대전지방법원 서산지원에서 도로교통법위반(음주운전)죄...


In [ ]:
ccnt = dataset[TARGET_SET][TARGET].value_counts()

class_counts = dataset[TARGET_SET][TARGET].value_counts().to_list() #43200, 4800
num_samples = sum(class_counts) # 48000 - 전체 데이터 갯수
labels = dataset[TARGET_SET][TARGET].to_list()

ccnt_map = {}
for index, row in dataset[TARGET_SET][TARGET].value_counts().iteritems():
    ccnt_map[index] = (float)(num_samples/row)

weights = []
for l in labels:
    weights.append(ccnt_map[l])

weights

<ipython-input-15-ef3fb0cd39fc>:8: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, row in dataset[TARGET_SET][TARGET].value_counts().iteritems():


[2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,
 2.236766121270452,


In [ ]:
len(weights)

4648

In [ ]:
test_d = None

In [ ]:
#from torchsampler import ImbalancedDatasetSampler

from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

class StatutesDataModule(pl.LightningDataModule):
    def __init__(self, column, trainset, statutes, tokenizer, datadf, batch_size=16, max_input_len=512):

        super().__init__()
        self.column = column
        self.trainset = trainset
        self.statutes = statutes
        self.tokenizer = tokenizer
        #self.data = data
        self.batch_size = batch_size
        self.max_input_len = max_input_len

        #divide train/valid/test set
        train, test = train_test_split(datadf, test_size=0.2, random_state=1)
        train, valid = train_test_split(train, test_size=0.2, random_state=1)
        self.data = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

        ###############################################
        #위에 데이터셋 class, 및 등등 선언되어있다 가정#
        ###############################################

        #class 0 : 43200개, class 1 : 4800개
        ccnt = train[TARGET].value_counts()

        class_counts = train[TARGET].value_counts().to_list() #43200, 4800
        num_samples = sum(class_counts) # 48000 - 전체 데이터 갯수
        labels = train[TARGET].to_list()

        #클래스별 가중치 부여 [48000/43200, 48000/4800] => class 1에 가중치 높게 부여하게 됨
        ccnt_map = {}
        for index, row in train[TARGET].value_counts().iteritems():
            ccnt_map[index] = (float)(num_samples/row)

        # 해당 데이터의 label에 해당되는 가중치
        weights = []
        for l in labels:
            weights.append(ccnt_map[l])

        self.trainsampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples)*1)

    def setup(self, stage):
        pass

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.data['train'], batch_size=self.batch_size, shuffle=False, collate_fn=self._collate_fn, num_workers=2, sampler=self.trainsampler)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.data['valid'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.data['test'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)

    def _collate_fn(self, batch):
        inputs = self.tokenizer([x[self.trainset] for x in batch], max_length=self.max_input_len, padding=True, truncation=True, return_tensors='pt')
        labels = [x[self.column] for x in batch]

        return inputs, labels

In [ ]:
from sklearn.model_selection import train_test_split
import datasets
import tensorflow as tf
from datasets import Dataset

def convertDFtoDS(df):
    return tf.data.Dataset.from_tensor_slices(dict(df))

backbone_card = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(backbone_card)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
import torch
import torchmetrics
from pprint import pprint

class StatutesClassifier(pl.LightningModule):
    def __init__(self, statutes, backbone, learning_rate=1e-5, mode='multi-class', approx_val=2):
        super().__init__()
        self.statutes = statutes
        self.backbone = backbone
        self.learning_rate = learning_rate
        self.mode = mode
        self.criterion = torch.nn.CrossEntropyLoss() if mode == 'multi-class' else torch.nn.BCEWithLogitsLoss()
        self.approx_val = approx_val
        self.judge = 0.5
        #self.criterion = torch.nn.MSELoss()
        #self.criterion = torch.nn.BCEWithLogitsLoss()
        # self.val_f1 = torchmetrics.F1Score(task='binary')
        # self.val_acc = torchmetrics.Accuracy(task='binary')
        # self.val_precision = torchmetrics.Precision(task='binary')
        # self.val_recall = torchmetrics.Recall(task='binary')
        # self.metric = torchmetrics.MetricCollection([self.val_f1, self.val_acc, self.val_precision, self.val_recall])

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def forward(self, batch):
        inputs, labels = batch
        logits = self.backbone(**inputs).logits
        targets = torch.zeros_like(logits)
        for i, label in enumerate(labels):
            label_id = self.statutes.index(label) if label < len(self.statutes) else 0
            # label_id = [(self.statutes.index(x) if x in self.statutes else 0) for x in label]
            #targets[i, label_id] = 1 / len(label_id) if self.mode == 'multi-class' else 1
            targets[i, label_id] = 1
        loss = self.criterion(logits, targets)

        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward(batch)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        return self._evaluation_step(batch)

    def validation_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs)
        # metric_out = self.metric.compute()
        # pprint(metric_out)
        # self.metric.reset()

    def test_step(self, batch, batch_idx):
        return self._evaluation_step(batch)

    def test_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs, True)

    def _evaluation_step(self, batch):
        loss, logits = self.forward(batch)
        _, gts = batch
        if self.mode == 'multi-class':
            prs = np.array(self.statutes)[logits.argmax(-1).cpu()].tolist()
        else:
            prs = [np.array(self.statutes)[np.where(logit.sigmoid().cpu() > self.judge)[0]] for logit in logits]

        # if isinstance(prs, list) == False:
        #     prs = [prs]

        return {'loss': loss.item(), 'gts': gts, 'prs': prs}

    def _evaluation_epoch_end(self, outputs, test=False):
        avg_loss = np.mean([x['loss'] for x in outputs])
        print(outputs)
        gts = reduce(lambda x,y: x + y, [x['gts'] for x in outputs], [])
        prs = reduce(lambda x,y: x + y, [x['prs'] for x in outputs], [])
        if self.mode == 'multi-class':
            #acc = sum([(pr in gt) for gt, pr in zip(gts, prs)]) / len(gts)
            acc = sum([abs(pr - gt) <= self.approx_val for gt, pr in zip(gts, prs)]) / len(gts)

            # True Positive, False Negative, False Positive
            TP = sum([1 for gt, pr in zip(gts, prs) if gt == 1 and pr == 1])
            FN = sum([1 for gt, pr in zip(gts, prs) if gt == 1 and pr == 0])
            FP = sum([1 for gt, pr in zip(gts, prs) if gt == 0 and pr == 1])

            recall = 0 if (TP+FN) == 0 else TP / (TP + FN)
            precision = 0 if (TP+FP) == 0 else TP / (TP + FP)
            f1 = 0 if (recall+precision) == 0 else 2 * (precision * recall) / (precision + recall)
            # acc = sum([abs(pr - gt[0]) <= self.approx_val for gt, pr in zip(gts, prs)]) / len(gts)

        else:
            acc = sum([(set(pr) == set(gt)) for gt, pr in zip(gts, prs)]) / len(gts)

        print('='*50)
        print(f'avg_loss: {avg_loss}')
        print(f'ACC: {acc}')
        print(f'REC: {recall}')
        print(f'PRC: {precision}')
        print(f'F1: {f1}')
        # print(self.metric(prs, gts))

        #target_ids = random.sample(range(len(gts)), 2)
        # target_ids = random.sample(range(len(prs)), 2)
        # print(gts)
        # print(prs)
        # print(f'target_ids: {target_ids}')
        # for target_id in target_ids:
        #     print(f'GT: {gts[target_id]}\t\t\tPR: {prs[target_id]}')

        # if self.mode == 'multi-class' and test:
        #     #df = pd.DataFrame(columns=['idx', 'gt', 'pr', 'etc'])
        #     #df = pd.DataFrame({'idx': pd.Series(dtype='int'), 'gt':pd.Series(dtype='int'), 'pr':pd.Series(dtype='int'),'etc':[]})
        #     df = pd.DataFrame({'idx': pd.Series(dtype='int'), 'gt':pd.Series(dtype='int'), 'pr':pd.Series(dtype='int')})
        #     print(data)
        #     for i, (gt, pr) in enumerate(zip(gts, prs)):
        #         if pr not in gt:
        #             # df = df.append(pd.DataFrame({'idx': [i],
        #             #                              'gt': ', '.join((str)(gt)),
        #             #                              'pr': pr,
        #             #                              'etc': data['test'][i]['etc']}),
        #             #                 ignore_index=True)
        #             df = df.append(pd.DataFrame({'idx': [i],
        #                                          'gt': ', '.join((str)(gt)),
        #                                          'pr': pr}),
        #                             ignore_index=True)
        #     df.to_csv('GT_vs_PR.csv', index=False)
    def configure_optimizers(self):
        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate}]
        optimizer = torch.optim.Adam(grouped_params, lr=self.learning_rate)
        #optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate) #       optimizer = torch.optim.AdamW(self.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
                                max_lr=self.learning_rate,
                                epochs=20,
                                optimizer=optimizer,
                                steps_per_epoch=int(1480 / 16),
                                pct_start=0.1,
                                div_factor=10,
                                final_div_factor=100,
                                base_momentum=0.90,
                                max_momentum=0.95,
                                )
        # return [optimizer], [scheduler]
        return optimizer

#    def configure_optimizers(self):
#        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate}]
#        optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate)
#
#        return {'optimizer': optimizer}

In [ ]:
# # coded by cat
# cfdxf,er?"
# ]\w
# ws"

In [ ]:
dataset

{'avail':           id  yooye  prison  money  \
 0        0.0    0.0     0.0    1.0   
 1        1.0    0.0     0.0    1.0   
 2        2.0    0.0     0.0    1.0   
 3        3.0    0.0     0.0    1.0   
 4        4.0    0.0     0.0    1.0   
 ...      ...    ...     ...    ...   
 4643  4643.0    0.0     1.0    0.0   
 4644  4644.0    0.0     0.0    1.0   
 4645  4645.0    0.0     1.0    0.0   
 4646  4646.0    0.0     1.0    0.0   
 4647  4647.0    0.0     0.0    1.0   
 
                                                      fy  
 0     레인지로버 스포츠 SDV6 승용차의 운전업무에 종사하는 피고인은 2021. 5. 1...  
 1     수원지방법원 성남지원에서 아동.청소년의성보호에관한법률위반(강간등)죄 등으로 소년부 ...  
 2     2020. 1. 24. 12:00경 혈중알코올농도 0.104%의 술에 취한 상태로 ...  
 3     피고인은 BMW 520d 승용차를 운전하는 업무에 종사하는 사람에게는 전방 및 좌우...  
 4     2021 6. 24. 19:35경 대전 대덕구 B에 있는 ‘C’ 앞 편도 1차선 도...  
 ...                                                 ...  
 4643  피고인은 2021. 7. 17. 02:10경 창원시 마산합포구 B에 있는 'C' 앞...  
 4644  충북 영동군 B 앞 도로부터 충북 영동군 C에 있는 D 앞 도로까지 약 2k

In [ ]:
import gc

def execTrain(dataset, target, trainset):
    #init cuda
    gc.collect()
    torch.cuda.empty_cache()

    # #divide train/valid/test set
    # train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    # train, valid = train_test_split(train, test_size=0.2, random_state=1)
    # d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

    #print(set(dataset[target].to_list()))

    # labels init

    labels = sorted(set(dataset[target].to_list()))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, dataset, batch_size=16, max_input_len=512)
    #data_module = StatutesDataModule(target, trainset, labels, tokenizer, dataset, batch_size=16, max_input_len=384)

    # model
    backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    model = StatutesClassifier(labels, backbone, learning_rate=2e-5, mode='multi-class', approx_val=TARGET_APPROX)

    # trainer
    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    # train and eval
    trainer.fit(model, data_module)
    trainer.test(model, data_module)

    return model

def execEval(dataset, target, trainset, modelpath):
    #init cuda
    gc.collect()
    torch.cuda.empty_cache()

    #divide train/valid/test set
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    train, valid = train_test_split(train, test_size=0.2, random_state=1)
    d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

    # labels init
    labels = sorted(set.union(*map(set, dataset[target])))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, d, batch_size=16, max_input_len=512)

    # model
    backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    model = StatutesClassifier(labels, backbone, learning_rate=2e-5, mode='multi-class')
    model = torch.load(modelpath)

    # trainer
    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    trainer.test(model, data_module)

## RUN

In [ ]:
# check variables
print(TARGET)
print(TARGET_SET)
print(TARGET_START)
print(TARGET_END)
print(TARGET_APPROX)
print(TARGET_OPT)
print(TRAIN_COLUMN)

money
avail
0
36
0
binary
fy


In [ ]:
dataset[TARGET_SET]

,id,yooye,prison,money,fy
0,0.0,0.0,0.0,1.0,레인지로버 스포츠 SDV6 승용차의 운전업무에 종사하는 피고인은 2021. 5. 1...
1,1.0,0.0,0.0,1.0,수원지방법원 성남지원에서 아동.청소년의성보호에관한법률위반(강간등)죄 등으로 소년부 ...
2,2.0,0.0,0.0,1.0,2020. 1. 24. 12:00경 혈중알코올농도 0.104%의 술에 취한 상태로 ...
3,3.0,0.0,0.0,1.0,피고인은 BMW 520d 승용차를 운전하는 업무에 종사하는 사람에게는 전방 및 좌우...
4,4.0,0.0,0.0,1.0,2021 6. 24. 19:35경 대전 대덕구 B에 있는 ‘C’ 앞 편도 1차선 도...
...,...,...,...,...,...
4643,4643.0,0.0,1.0,0.0,피고인은 2021. 7. 17. 02:10경 창원시 마산합포구 B에 있는 'C' 앞...
4644,4644.0,0.0,0.0,1.0,충북 영동군 B 앞 도로부터 충북 영동군 C에 있는 D 앞 도로까지 약 2km 구간...
4645,4645.0,0.0,1.0,0.0,2011. 2. 25. 창원지방법원 거창지원에서 도로교통법위반(음주운전)죄로 벌금 ...
4646,4646.0,0.0,1.0,0.0,피고인은 2009. 7. 16. 대전지방법원 서산지원에서 도로교통법위반(음주운전)죄...


In [ ]:
type(dataset['avail'])

pandas.core.frame.DataFrame

In [ ]:
# 양형의 이유 있는 데이터 중 etc로 prison classification, MSE
model = execTrain(dataset[TARGET_SET], TARGET, TRAIN_COLUMN)
torch.save(model, f'{MODEL_DIR}/{TARGET}_{TARGET_SET}_{TRAIN_COLUMN}_{TARGET_START}_{TARGET_END}_model_imb.pt')

<ipython-input-18-db2bc99a3741>:35: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, row in train[TARGET].value_counts().iteritems():
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be e

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


[{'loss': 0.6723411083221436, 'gts': [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0], 'prs': [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, {'loss': 0.6920182704925537, 'gts': [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0], 'prs': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]}]
avg_loss: 0.6821796894073486
ACC: 0.578125
REC: 0.13793103448275862
PRC: 0.6666666666666666
F1: 0.2285714285714286


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

[{'loss': 0.6735233068466187, 'gts': [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0], 'prs': [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]}, {'loss': 0.5181069374084473, 'gts': [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0], 'prs': [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]}, {'loss': 0.49914222955703735, 'gts': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], 'prs': [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

[{'loss': 0.5990532636642456, 'gts': [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0], 'prs': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]}, {'loss': 0.4176728129386902, 'gts': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0], 'prs': [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]}, {'loss': 0.6166002154350281, 'gts': [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0], 'prs': [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.